<a href="https://colab.research.google.com/github/KarimDataMaster/AirFlow/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%9F%D0%B8%D1%88%D0%B5%D0%BC_%D0%BF%D0%B0%D0%B9%D0%BF%D0%BB%D0%B0%D0%B9%D0%BD_%D1%81_%D0%BD%D1%83%D0%BB%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Скрипт для решения задачи автоматизации отчетности

In [8]:
import pandas as pd
import sqlite3

CON = sqlite3.connect('example.db')

In [9]:
# Выгрузка данных с сайта
def extract_data(url):
    """ Extract CSV
    """
    return pd.read_csv(url)

# Группировка данных
def transform_data(data, group, agreg):
    """ Group by data
    """
    return data.groupby(group).agg(agreg).reset_index()

# Загрузка в базу данных
def load_data(data, table_name, conn=CON):
    """ Load to DB
    """
    data["insert_time"] = pd.to_datetime("now")
    data.to_sql(table_name, conn, if_exists='replace', index=False)

# Отправка данных на почту
def send_email(data, cred, host, port, to, From):
    """ Send to email
    """ 
    _send_email(data=data, cred=cred, host=host, to=to, From=From, port=port)

In [ ]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib

HOST = "smtp.yandex.ru"
TO = "rabacc2022@yandex.ru"
FROM = "rabacc2022@yandex.ru"


def html_pretty(df):
    """ Pretty html dataframe
    """
    return """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())

def _send_email(data, cred, host, port, to, From):
    """ Send DF to email
    """

    msg = MIMEMultipart()
    part = MIMEText(html_pretty(data), 'html')
    msg.attach(part)

    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(cred[0], cred[1])
    server.sendmail(From, to, msg.as_string())
    server.quit()

In [ ]:
# Extract
data = extract_data("https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data/data.csv")

In [ ]:
# Transform
data = transform_data(data, group=['A', 'B', 'C'], agreg={"D": sum})

In [ ]:
# Load to DB
load_data(data, "table")

In [ ]:
# Send Email
send_email(data, cred=("rabacc2022", "Karim789-"), host=HOST, port=587, to=TO, From=FROM)